## Test 1 - New data format with seq_len incorporated into dataset code


In [134]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
# Import comet_ml in the top of your file
from comet_ml import Experiment
##Needs to be imported before sklearn
from sklearn.model_selection import train_test_split
import sklearn

import torch
import torch.utils.data
from torch.utils.data import Dataset, TensorDataset, DataLoader #as DataLoaderBase
from torch import nn, optim, sigmoid
from torch.nn import functional as F
from torchvision import datasets, transforms
from torch.nn import modules
from torch.nn.modules import loss

from torch.autograd import Variable

#from torchaudio import transforms
#from torchaudio import Datasets

import os
import sys
import shutil
from glob import glob
import datetime
import re
from tqdm import tqdm
import time

import json
from pprint import pprint
#from torchviz import make_dot, make_dot_from_trace


In [135]:
torch.__version__

'0.4.1'

In [136]:
# Check device type


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [137]:

def data_creator(T = 20, L = 10000, N = 1000):
    """
    Simple sine wave creator. Used to build a large dataset of sine waves.
    
    
    """
    np.random.seed(2)



    x = np.empty((N, L), 'int64')
    x[:] = np.array(range(L)) + np.random.randint(-4 * T, 4 * T, N).reshape(N, 1)
    data = np.sin(x / 1.0 / T).astype('float64')
    data = sklearn.preprocessing.normalize(data)

    torch.save(data, open('traindata.pt', 'wb'))

In [138]:

data_creator()

In [139]:
class EncoderRNN(nn.Module):
    """
    Code adapted from ----
    
    """
    def __init__(self, input_size, hidden_size, num_layers, isCuda):
        super(EncoderRNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.isCuda = isCuda
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.relu = nn.ReLU()
        
        #initialize weights
        nn.init.xavier_uniform(self.lstm.weight_ih_l0, gain=np.sqrt(2))
        nn.init.xavier_uniform(self.lstm.weight_hh_l0, gain=np.sqrt(2))

    def forward(self, input):
        tt = torch.cuda if self.isCuda else torch
        h0 = Variable(tt.FloatTensor(self.num_layers, input.size(0), self.hidden_size))
        c0 = Variable(tt.FloatTensor(self.num_layers, input.size(0), self.hidden_size))
        encoded_input, hidden = self.lstm(input, (h0, c0))
        encoded_input = self.relu(encoded_input)
        return encoded_input

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, num_layers, isCuda):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_layers
        
        self.isCuda = isCuda
        self.lstm = nn.LSTM(hidden_size, output_size, num_layers, batch_first=True)
        #self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
        #initialize weights
        nn.init.xavier_uniform(self.lstm.weight_ih_l0, gain=np.sqrt(2))
        nn.init.xavier_uniform(self.lstm.weight_hh_l0, gain=np.sqrt(2))
        
    def forward(self, encoded_input):
        tt = torch.cuda if self.isCuda else torch
        h0 = Variable(tt.FloatTensor(self.num_layers, encoded_input.size(0), self.output_size))
        c0 = Variable(tt.FloatTensor(self.num_layers, encoded_input.size(0), self.output_size))
        decoded_output, hidden = self.lstm(encoded_input, (h0, c0))
        decoded_output = self.sigmoid(decoded_output)
        return decoded_output

class LSTMAE(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, isCuda):
        super(LSTMAE, self).__init__()
        self.encoder = EncoderRNN(input_size, hidden_size, num_layers, isCuda)
        self.decoder = DecoderRNN(hidden_size, input_size, num_layers, isCuda)
        
    def forward(self, input):
        encoded_input = self.encoder(input)
        decoded_output = self.decoder(encoded_input)
        return decoded_output

In [140]:
class sineDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, seq_len, overlap):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.data = torch.load('traindata.pt')
        self.label = []
        self.seq_len = seq_len
        self.overlap = overlap
        vals = len(self.data)
        for val in range(vals):
            self.label.append(val)
        if torch.cuda.is_available():
            self.data, self.label = map(torch.cuda.FloatTensor, (self.data, self.label))
        else:
            self.data, self.label = map(torch.FloatTensor, (self.data, self.label))

        self.data = self.window_mask()
        self.data = torch.as_tensor(self.data)
        self.data.permute(2,1,0 )
        print("shape = ", self.data.shape)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]
    
    def window_mask(self):
        datavals = []
        results = []
        self.data= torch.tensor(self.data)
        start_val = 0
        occurences = int(len(self.data)/(self.seq_len*self.overlap))
        
        for rows in range(len(self.data)):

            start_val = 0
            for i in range(occurences-1):
                
                    value = self.data[rows,start_val:(start_val+self.seq_len)]
                    
                    start_val += int(self.seq_len*(1 - self.overlap))
                    datavals.append(value.unsqueeze(0))

            var = torch.cat(datavals)        
            results.append(var.unsqueeze(0))
            
            datavals = []
        result = torch.cat(results)
        result = result.permute(2,1,0)
        print(result.shape)
        print(result)
        return result
    
        

In [141]:
def get_data(train_dataset, test_dataset, bs):
    return (DataLoader(train_dataset, batch_size = batch_size),
    DataLoader(test_dataset, batch_size = batch_size * 2))

In [142]:
#hyperparameters

with open('config_file.json') as f:
    data = json.load(f)

pprint(data)

hyperparameters = data["hyperparams"]
optimiser = data["optimizer"]
trainer = data["trainer"]
loss_name = data["loss"]

{'arch': 'seabedmodel',
 'cuda': True,
 'data_loader': {'batch_size': 32,
                 'data_dir': 'datasets/',
                 'shuffle': True,
                 'type': 'ImageDataLoader'},
 'gpu': 0,
 'hyperparams': {'batch_size': 16,
                 'hidden_dim': 400,
                 'input_size': 1000,
                 'latent': 5,
                 'num_layers': 3,
                 'overlap': 0.5,
                 'seq_len': 100},
 'loss': 'BCEWithLogitsLoss',
 'loss_args': {'reduction': 'elementwise_mean'},
 'metrics': ['my_metric', 'my_metric2'],
 'model': {},
 'name': 'Autoencoder',
 'optimizer': {'lr': 0.001, 'weight_decay': 0},
 'optimizer_type': 'Adam',
 'trainer': {'epochs': 1000,
             'monitor': 'val_loss',
             'monitor_mode': 'min',
             'save_dir': 'saved/',
             'save_freq': 1,
             'verbosity': 2},
 'validation': {'shuffle': True, 'test_split': 0.2, 'validation_split': 0.1},
 'visualization': {'log_dir': 'saved/runs', 'tens

In [143]:
 
full_dataset = sineDataset( hyperparameters["seq_len"], hyperparameters["overlap"])
full_dataset
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])
train_dl, test_dl = get_data(train_dataset, test_dataset, hyperparameters["batch_size"])
print(train_dl)

torch.Size([100, 19, 1000])
tensor([[[ 0.0015, -0.0055, -0.0034,  ..., -0.0126,  0.0041, -0.0132],
         [-0.0096,  0.0122, -0.0055,  ...,  0.0139, -0.0114,  0.0136],
         [ 0.0139, -0.0141,  0.0122,  ..., -0.0097,  0.0141, -0.0087],
         ...,
         [-0.0115,  0.0134, -0.0080,  ...,  0.0132, -0.0128,  0.0126],
         [ 0.0141, -0.0135,  0.0134,  ..., -0.0075,  0.0139, -0.0063],
         [-0.0112,  0.0082, -0.0135,  ..., -0.0012, -0.0094, -0.0026]],

        [[ 0.0008, -0.0049, -0.0041,  ..., -0.0123,  0.0034, -0.0129],
         [-0.0091,  0.0118, -0.0049,  ...,  0.0140, -0.0109,  0.0138],
         [ 0.0138, -0.0141,  0.0118,  ..., -0.0102,  0.0141, -0.0092],
         ...,
         [-0.0111,  0.0131, -0.0074,  ...,  0.0134, -0.0125,  0.0129],
         [ 0.0141, -0.0137,  0.0131,  ..., -0.0081,  0.0140, -0.0069],
         [-0.0116,  0.0088, -0.0137,  ..., -0.0005, -0.0099, -0.0019]],

        [[ 0.0001, -0.0042, -0.0047,  ..., -0.0119,  0.0027, -0.0126],
         [-0.0086

In [144]:

def get_model():
    
    #model = EncoderRNN(input_dim= input_dim, hidden_dim= hidden_dim).to(device)
    model = LSTMAE(input_size = hyperparameters["input_size"], hidden_size = hyperparameters["hidden_dim"], num_layers = hyperparameters["num_layers"], isCuda = False)

    
    return model, optim.Adam(model.parameters(), lr = lr)

In [145]:
def loss_function(output, x): 
    loss_fn = getattr(loss, loss_name)
    print(loss_fn)
    loss_fn = F.binary_cross_entropy
    return loss_fn(output, x)


In [146]:
def loss_batch(model, loss_function, xb, opt=None): 
    output = model(xb) 
    print(output)
    loss = loss_function(output, xb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)


In [147]:
def fit(epochs, model, loss_func, opt, train_dl, test_dl):
    for epoch in tqdm(range(epochs)):
        model.train()
        
        for xb in train_dl:
            
            print(xb.shape)
            #xb = xb.reshape(1,batch_size,input_size).to(device)  
           
            if torch.cuda.is_available():
                xb = Variable(xb)
            else:
                xb = Variable(xb)
            
            """seq_len, batch_size, input_size
            input_size is 1000 seperate strings.
            batch_size = 16
            seq_len can be anything....
            
            
            """
            loss_batch(model, loss_func, xb, opt)
            model.eval()
            with torch.no_grad():
                losses,nums = zip(*[loss_batch(model, loss_func, xb)
                                    for xb in test_dl])
            val_loss = np.sum(np.multiply(losses,nums)) / np.sum(nums)
        print(epoch, val_loss)

In [148]:

model, opt = get_model()
#x = torch.randn(1, 10000)
#make_dot(model(x), params=dict(model.named_parameters()))

       
    

C:\Users\Tim\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
C:\Users\Tim\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
C:\Users\Tim\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
C:\Users\Tim\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


In [ ]:
fit(epochs, model, loss_function, opt, train_dl, test_dl)


  0%|                                                                                        | 0/1000 [00:00<?, ?it/s]

torch.Size([32, 19, 1000])
tensor([[[0.5033, 0.4984, 0.5008,  ..., 0.4984, 0.4989, 0.4966],
         [0.5004, 0.4968, 0.5019,  ..., 0.4994, 0.4978, 0.4963],
         [0.4982, 0.4962, 0.5022,  ..., 0.4998, 0.4973, 0.4965],
         ...,
         [0.4942, 0.4969, 0.4996,  ..., 0.4993, 0.4970, 0.5012],
         [0.4943, 0.4969, 0.4996,  ..., 0.4993, 0.4970, 0.5012],
         [0.4943, 0.4969, 0.4996,  ..., 0.4993, 0.4969, 0.5012]],

        [[0.4959, 0.4957, 0.4953,  ..., 0.4981, 0.4966, 0.4995],
         [0.4957, 0.4964, 0.4971,  ..., 0.4991, 0.4967, 0.4996],
         [0.4954, 0.4967, 0.4982,  ..., 0.4994, 0.4967, 0.4999],
         ...,
         [0.4943, 0.4969, 0.4996,  ..., 0.4993, 0.4970, 0.5012],
         [0.4943, 0.4969, 0.4996,  ..., 0.4993, 0.4970, 0.5012],
         [0.4943, 0.4969, 0.4996,  ..., 0.4993, 0.4969, 0.5012]],

        [[0.5021, 0.4988, 0.4992,  ..., 0.4981, 0.4995, 0.4982],
         [0.4988, 0.4976, 0.4992,  ..., 0.4989, 0.4986, 0.4994],
         [0.4969, 0.4971, 0.499

tensor([[[0.4658, 0.4753, 0.4725,  ..., 0.4747, 0.4623, 0.4940],
         [0.3505, 0.3622, 0.3561,  ..., 0.3596, 0.3479, 0.4940],
         [0.2776, 0.2799, 0.2790,  ..., 0.2798, 0.2770, 0.4999],
         ...,
         [0.2690, 0.2690, 0.2690,  ..., 0.2690, 0.2690, 0.5000],
         [0.2690, 0.2690, 0.2690,  ..., 0.2690, 0.2690, 0.5000],
         [0.2690, 0.2690, 0.2690,  ..., 0.2690, 0.2690, 0.5000]],

        [[0.4648, 0.4631, 0.4635,  ..., 0.4672, 0.4665, 0.4899],
         [0.3487, 0.3532, 0.3494,  ..., 0.3534, 0.3475, 0.4940],
         [0.2773, 0.2782, 0.2778,  ..., 0.2787, 0.2772, 0.4999],
         ...,
         [0.2690, 0.2690, 0.2690,  ..., 0.2690, 0.2690, 0.5000],
         [0.2690, 0.2690, 0.2690,  ..., 0.2690, 0.2690, 0.5000],
         [0.2690, 0.2690, 0.2690,  ..., 0.2690, 0.2690, 0.5000]],

        [[0.4647, 0.4788, 0.4696,  ..., 0.4701, 0.4598, 0.4941],
         [0.3465, 0.3593, 0.3503,  ..., 0.3526, 0.3422, 0.4946],
         [0.2769, 0.2795, 0.2781,  ..., 0.2787, 0.2763, 0.

  0%|                                                                              | 1/1000 [00:17<4:54:09, 17.67s/it]

torch.Size([32, 19, 1000])
tensor([[[0.4480, 0.4966, 0.5047,  ..., 0.4495, 0.4618, 0.4663],
         [0.2905, 0.3165, 0.3221,  ..., 0.2933, 0.2924, 0.5000],
         [0.2719, 0.2758, 0.2766,  ..., 0.2723, 0.2722, 0.5000],
         ...,
         [0.2689, 0.2689, 0.2689,  ..., 0.2689, 0.2689, 0.5000],
         [0.2689, 0.2689, 0.2689,  ..., 0.2689, 0.2689, 0.5000],
         [0.2689, 0.2689, 0.2689,  ..., 0.2689, 0.2689, 0.5000]],

        [[0.4746, 0.4941, 0.4677,  ..., 0.4781, 0.4781, 0.4469],
         [0.3000, 0.3140, 0.2981,  ..., 0.2982, 0.2998, 0.5000],
         [0.2732, 0.2753, 0.2730,  ..., 0.2730, 0.2732, 0.5000],
         ...,
         [0.2689, 0.2689, 0.2689,  ..., 0.2689, 0.2689, 0.5000],
         [0.2689, 0.2689, 0.2689,  ..., 0.2689, 0.2689, 0.5000],
         [0.2689, 0.2689, 0.2689,  ..., 0.2689, 0.2689, 0.5000]],

        [[0.4475, 0.4937, 0.5017,  ..., 0.4474, 0.4605, 0.4669],
         [0.2902, 0.3147, 0.3197,  ..., 0.2926, 0.2922, 0.5000],
         [0.2719, 0.2755, 0.276

...

  6%|████▊                                                                       | 64/1000 [54:16<13:13:40, 50.88s/it]

torch.Size([32, 19, 1000])
tensor([[[0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.2689],
         [0.3183, 0.3183, 0.3183,  ..., 0.3183, 0.3183, 0.5000],
         [0.2761, 0.2761, 0.2761,  ..., 0.2761, 0.2762, 0.5000],
         ...,
         [0.2689, 0.2689, 0.2689,  ..., 0.2689, 0.2689, 0.5000],
         [0.2689, 0.2689, 0.2689,  ..., 0.2689, 0.2689, 0.5000],
         [0.2689, 0.2689, 0.2689,  ..., 0.2689, 0.2689, 0.5000]],

        [[0.5000, 0.3183, 0.5000,  ..., 0.2689, 0.3183, 0.2689],
         [0.3064, 0.2761, 0.3183,  ..., 0.2689, 0.3114, 0.5000],
         [0.2743, 0.2699, 0.2761,  ..., 0.2689, 0.2750, 0.5000],
         ...,
         [0.2689, 0.2689, 0.2689,  ..., 0.2689, 0.2689, 0.5000],
         [0.2689, 0.2689, 0.2689,  ..., 0.2689, 0.2689, 0.5000],
         [0.2689, 0.2689, 0.2689,  ..., 0.2689, 0.2689, 0.5000]],

        [[0.2689, 0.5000, 0.3183,  ..., 0.2689, 0.3183, 0.2689],
         [0.2689, 0.7311, 0.2835,  ..., 0.2689, 0.2761, 0.5000],
         [0.2689, 0.7311, 0.270

         [0.2689, 0.2689, 0.2689,  ..., 0.2689, 0.2689, 0.5000]]])
<class 'torch.nn.modules.loss.BCEWithLogitsLoss'>
torch.Size([16, 19, 1000])
tensor([[[0.3183, 0.5000, 0.5000,  ..., 0.3183, 0.3183, 0.3183],
         [0.2761, 0.2761, 0.3183,  ..., 0.2761, 0.2761, 0.5000],
         [0.2699, 0.2699, 0.2761,  ..., 0.2699, 0.2699, 0.5000],
         ...,
         [0.2689, 0.2689, 0.2689,  ..., 0.2689, 0.2689, 0.5000],
         [0.2689, 0.2689, 0.2689,  ..., 0.2689, 0.2689, 0.5000],
         [0.2689, 0.2689, 0.2689,  ..., 0.2689, 0.2689, 0.5000]],

        [[0.5000, 0.5000, 0.3183,  ..., 0.5000, 0.5000, 0.7311],
         [0.3183, 0.2761, 0.2761,  ..., 0.3183, 0.3183, 0.5000],
         [0.2761, 0.2699, 0.2699,  ..., 0.2761, 0.2761, 0.5000],
         ...,
         [0.2689, 0.2689, 0.2689,  ..., 0.2689, 0.2689, 0.5000],
         [0.2689, 0.2689, 0.2689,  ..., 0.2689, 0.2689, 0.5000],
         [0.2689, 0.2689, 0.2689,  ..., 0.2689, 0.2689, 0.5000]],

        [[0.3183, 0.5000, 0.5000,  ..., 0.31

  6%|████▉                                                                       | 65/1000 [55:28<13:17:53, 51.20s/it]

torch.Size([32, 19, 1000])
tensor([[[0.5000, 0.5000, 0.5000,  ..., 0.5000, 0.5000, 0.2689],
         [0.3183, 0.3183, 0.3183,  ..., 0.3183, 0.3183, 0.5000],
         [0.2761, 0.2761, 0.2761,  ..., 0.2761, 0.2762, 0.5000],
         ...,
         [0.2689, 0.2689, 0.2689,  ..., 0.2689, 0.2689, 0.5000],
         [0.2689, 0.2689, 0.2689,  ..., 0.2689, 0.2689, 0.5000],
         [0.2689, 0.2689, 0.2689,  ..., 0.2689, 0.2689, 0.5000]],

        [[0.5000, 0.3183, 0.5000,  ..., 0.2689, 0.3183, 0.2689],
         [0.3130, 0.2761, 0.3183,  ..., 0.2689, 0.3121, 0.5000],
         [0.2753, 0.2699, 0.2761,  ..., 0.2689, 0.2751, 0.5000],
         ...,
         [0.2689, 0.2689, 0.2689,  ..., 0.2689, 0.2689, 0.5000],
         [0.2689, 0.2689, 0.2689,  ..., 0.2689, 0.2689, 0.5000],
         [0.2689, 0.2689, 0.2689,  ..., 0.2689, 0.2689, 0.5000]],

        [[0.2689, 0.5000, 0.3183,  ..., 0.2689, 0.3183, 0.2689],
         [0.2689, 0.7311, 0.2830,  ..., 0.2689, 0.2761, 0.5000],
         [0.2689, 0.7311, 0.270